In [2]:
import cv2
import numpy as np
import requests
from io import BytesIO
import json
import matplotlib.pyplot as plt
import random

# Input from user
input_file_name = input('Input File Name (e.g., Bio_Data.json or Rainforest_Data.json): ')
input_logo_file = input('Which Logo to detect (e.g., Bio_logo.png or rainforest_logo.png): ')

def download_image(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = np.asarray(bytearray(response.content), dtype=np.uint8)
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            return image
        else:
            print("Failed to download image from URL:", url)
            return None
    except Exception as e:
        print("Error downloading image from URL:", url)
        print(e)
        return None

def detect_sift_keypoints(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def match_keypoints(descriptors1, descriptors2):
    bf_matcher = cv2.BFMatcher()
    matches = bf_matcher.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.40 * n.distance:  # Adjust the threshold as needed, 0.40 standard for vegan
            good_matches.append(m)
    return good_matches

def verify_keypoint_matches(image1, image2, keypoints1, keypoints2, matches):
    verified_matches = []
    for match in matches:
        kp1 = keypoints1[match.queryIdx]
        kp2 = keypoints2[match.trainIdx]
        
        x1, y1 = int(kp1.pt[0]), int(kp1.pt[1])
        x2, y2 = int(kp2.pt[0]), int(kp2.pt[1])
        roi1 = image1[max(y1-10, 0):y1+10, max(x1-10, 0):x1+10]
        roi2 = image2[max(y2-10, 0):y2+10, max(x2-10, 0):x2+10]

        contours1, _ = cv2.findContours(roi1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours2, _ = cv2.findContours(roi2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        area1 = cv2.contourArea(contours1[0]) if contours1 else 0
        area2 = cv2.contourArea(contours2[0]) if contours2 else 0

        if area1 > 0 and area2 > 0 and abs(area1 - area2) < 5:  # Ensure valid contours and adjust threshold as needed
            verified_matches.append(match)
    
    return verified_matches

def classify_product(images, logo_images):
    logo_count = []
    total_verified_matches = []
    for i in range(len(images)):
        logo_detected_count = 0
        product_image = images[i]
        logo_image = logo_images[i]
        
        keypoints_product, descriptors_product = detect_sift_keypoints(product_image)
        keypoints_logo, descriptors_logo = detect_sift_keypoints(logo_image)
        matches = match_keypoints(descriptors_product, descriptors_logo)
        verified_matches = verify_keypoint_matches(product_image, logo_image, keypoints_product, keypoints_logo, matches)
        
        total_verified_matches.append(len(verified_matches))

        if len(verified_matches) >= 1:
            logo_detected_count += 1
        logo_count.append(logo_detected_count)
        
    print('Total verified matches:', sum(total_verified_matches))
    print ('Total RGB color matches out of 3 :', sum(logo_count))
    if (sum(logo_count) >= 2) and (sum(total_verified_matches) >= threshold_matches):
        return (detecting_name, sum(logo_count), sum(total_verified_matches))
    else:
        return ('Non-'+detecting_name, sum(logo_count), sum(total_verified_matches))

def read_product_image(url):
    product_image = download_image(url)
    logo_image = cv2.imread(str(input_logo_file))
    return (product_image, logo_image)

def split_color_channels(product_image, logo_image):
    product_channels = cv2.split(product_image)
    logo_channels = cv2.split(logo_image)
    return (product_channels, logo_channels)

def main_function(data):
    for i, entry in enumerate(data):
        print('Processing URL number:', i+1)
        url = entry['Product_Image']
        print(url)
        global detecting_name
        global threshold_matches
        product_image, logo_image = read_product_image(url)
        if input_file_name == 'Bio_Data.json':
            column_name, detecting_name, threshold_matches = 'IsBio', 'Bio', 1
        elif input_file_name == 'Rainforest_Data.json':
            column_name, detecting_name, threshold_matches = 'IsRainforest', 'Rainforest', 30 
            
        if product_image is None or logo_image is None:
            print("Error: Unable to download or process image")
            continue

        product_channels, logo_channels = split_color_channels(product_image, logo_image)

        if product_channels is None or logo_channels is None:
            print("Error: Unable to split color channels")
            continue

        classification, color_match, verified_match_count = classify_product(product_channels, logo_channels)       
        print('Detected:', classification, '    Actual:', entry[column_name])
        entry['SIFT_ContourArea_detection'] = classification
        entry['SIFT_ContourArea_detection_colormatch'] = color_match
        entry['SIFT_ContourArea_detection_verifymatch'] = verified_match_count
        print('*' * 100)


# Read JSON file line by line and append each JSON object to combined_data list
with open(input_file_name, 'r', encoding='utf-8') as json_file:
    combined_data = json.load(json_file)

main_function(combined_data)

with open('Result_Detection.json', 'w') as json_file:
    json.dump(combined_data, json_file, indent=4)


Input File Name (e.g., Bio_Data.json or Rainforest_Data.json): Bio_Data.json
Which Logo to detect (e.g., Bio_logo.png or rainforest_logo.png): Bio_logo.png
Processing URL number: 1
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2023/kw51/produkte/1009614_51-2023_EhrmannGrandDessert-Vanille_ON_Comp.png/_jcr_content/renditions/opt.1250w.png.res/1700731629013/opt.1250w.png
Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-Bio
****************************************************************************************************
Processing URL number: 2
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2024/kw02/produkte/1002315_02-2024_BioLeinoel-FS_ON.png/_jcr_content/renditions/opt.1250w.png.res/1702543068979/opt.1250w.png
Total verified matches: 11
Total RGB color matches out of 3 : 2
Detected: Bio     Actual: Bio
****************************************************************************************************
Processi

Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-Bio
****************************************************************************************************
Processing URL number: 21
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2024/kw13/produkte/1011534_13-2024_FunkyAmericanVegan-FS03_ON_Comp.png/_jcr_content/renditions/opt.1250w.png.res/1709540750644/opt.1250w.png
Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-Bio
****************************************************************************************************
Processing URL number: 22
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2023/sommersortiment/8350_12-2023_COLOMBARDSAUVIGNONVIN-02_ON.png/_jcr_content/renditions/opt.1250w.png.res/1680257403103/opt.1250w.png
Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-Bio
************************************************

Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-Bio
****************************************************************************************************
Processing URL number: 41
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2024/kw02/produkte/1030056_02-2024_MYWAYBioMandelKokosnussdrink-02_ON_Com.png/_jcr_content/renditions/opt.1250w.png.res/1702547757025/opt.1250w.png
Total verified matches: 19
Total RGB color matches out of 3 : 3
Detected: Bio     Actual: Bio
****************************************************************************************************
Processing URL number: 42
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2022/kw20/produkte/1016069_20-2022_BABYPFLEGETCHER-03_Comp.png/_jcr_content/renditions/opt.1250w.png.res/1650868616802/opt.1250w.png
Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-Bio
***************************************************

Total verified matches: 32
Total RGB color matches out of 3 : 2
Detected: Bio     Actual: Bio
****************************************************************************************************
Processing URL number: 61
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2024/kw01/produkte/1007944_01-2024_Tomatensauce-Klassik_ON_Comp.png/_jcr_content/renditions/opt.1250w.png.res/1702901251405/opt.1250w.png
Total verified matches: 29
Total RGB color matches out of 3 : 3
Detected: Bio     Actual: Bio
****************************************************************************************************
Processing URL number: 62
https://www.aldi-nord.de/content/dam/aldi/germany/produkte/milchprodukte/1002013_46-2021_BIOSCHLAGSAHNE30200G-01_ON.png/_jcr_content/renditions/opt.1250w.png.res/1630408046599/opt.1250w.png
Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Bio
**************************************************************************

Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Bio
****************************************************************************************************
Processing URL number: 81
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2021/kw01/produkte/1002316_01-2021_SK_BioChiaSamen_FS_0536_ON.png/_jcr_content/renditions/opt.1250w.png.res/1607428139847/opt.1250w.png
Total verified matches: 11
Total RGB color matches out of 3 : 3
Detected: Bio     Actual: Bio
****************************************************************************************************
Processing URL number: 82
https://www.aldi-nord.de/content/dam/aldi/germany/produkte/wurst/6405_50-2022_HAUCHFEINEXCELLENCEQS125150G-Schinkenwurst_Comp.png/_jcr_content/renditions/opt.1250w.png.res/1671184868049/opt.1250w.png
Failed to download image from URL: https://www.aldi-nord.de/content/dam/aldi/germany/produkte/wurst/6405_50-2022_HAUCHFEINEXCELLENCEQS125150G-Schinkenwurst_Comp

Total verified matches: 21
Total RGB color matches out of 3 : 3
Detected: Bio     Actual: Bio
****************************************************************************************************
Processing URL number: 100
https://www.aldi-nord.de/content/dam/aldi/germany/produkte/tiefgekuehltes/4501_38-2021_BioHaferflocken-01_ON.png/_jcr_content/renditions/opt.1250w.png.res/1632121309035/opt.1250w.png
Total verified matches: 22
Total RGB color matches out of 3 : 2
Detected: Bio     Actual: Bio
****************************************************************************************************
Processing URL number: 101
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2024/kw11/produkte/1029019_11-2024_BioVanilleExtrakt-GutBio_ON.png/_jcr_content/renditions/opt.1250w.png.res/1708455017456/opt.1250w.png
Total verified matches: 30
Total RGB color matches out of 3 : 2
Detected: Bio     Actual: Bio
***********************************************************************************

Failed to download image from URL: https://www.aldi-nord.de/content/dam/aldi/germany/produkte/milchprodukte/6550_46-2021_JOGHURTAUFFRUCHT-Erdbeere_Comp.png/_jcr_content/renditions/opt.1250w.png.res/1639482536296/opt.1250w.png
Error: Unable to download or process image
Processing URL number: 120
https://www.aldi-nord.de/content/dam/aldi/germany/produkte/milchprodukte/1019000_48-2022_HFettarmeMilch-01_ON.png/_jcr_content/renditions/opt.1250w.png.res/1669969657429/opt.1250w.png
Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-Bio
****************************************************************************************************
Processing URL number: 121
https://www.aldi-nord.de/content/dam/aldi/germany/angebote/2022/kw28/produkte/1385_28-2022_GegarterSchweineb-02_ON_Comp.png/_jcr_content/renditions/opt.1250w.png.res/1656689375695/opt.1250w.png
Total verified matches: 0
Total RGB color matches out of 3 : 0
Detected: Non-Bio     Actual: Non-